In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re


julia_roberts_wiki_url = 'https://en.wikipedia.org/wiki/Julia_Roberts_filmography'
page = urlopen(julia_roberts_wiki_url)
soup = BeautifulSoup(page)
table=soup.find('table', class_='wikitable plainrowheaders sortable')
base_url = 'https://en.wikipedia.org/'
awards_prefix = 'https://en.wikipedia.org/wiki/List_of_awards_and_nominations_received_by_'



years=[]
names=[]
countries=[]
awards=[]

# get all available links for all movie pages
movies_url=[]
for row in table.findAll("tr"):
    headers=row.findAll('th')
    if(len(headers)==1):
        link = headers[0].find(href=True)
        if(link is not None):
            movies_url.append(base_url + link.get('href'))
            
            
def getSoupFromUrl(url):
    page = urlopen(url)
    return BeautifulSoup(page)

def getYearOfBirth(born_details):
    dataRegex = re.compile(r'\d\d\d\d')
    for text in born_details:
        match = dataRegex.search(text)
        if match is not None:
          group = match.group()
          return group
        
    return 'Year not found'
        
def getPlaceOfBirth(row):
    div = row.find('div', class_='birthplace')
    if div is not None:
        return div.find(text=True)
    else:
        #msometimes there is only link to the country
        country = row.find(href=True)
        if(country is not None):
            return country.find(text=True)
        else:
            return "Country Not Found"
        
def getNumOfAwards(name):
    actor_awards = awards_prefix + name.replace(' ', '_')
    try:
        awards_soup = BeautifulSoup(urlopen(actor_awards), "lxml")
        awards_list = awards_soup.find_all("td", class_ = "yes table-yes2", text = re.compile('Won'))
        num_of_awards = len(awards_list)
    except:
        num_of_awards = 'na'
    return num_of_awards
        
def setArraysWithActorsData(actor_url):
    try:
        soup = getSoupFromUrl(actor_url)
        table = soup.find('table', class_='infobox')
        if table is not None:
            year=False
            country=False
            name=False
            #find only the name of the actor
            first_row = table.findAll("tr")[0]
            header = first_row.findAll('th')
            name = header[0].find(text=True)
            if name is None:
                name='Name not found'

            #find born details
            for row in table.findAll("tr"):
                if row.find(text=True)=='Born':
                    born_details =row.findAll('td')[0].findAll(text=True)
                    year = getYearOfBirth(born_details)
                    country = getPlaceOfBirth(row)
                    a_awards = getNumOfAwards(name)
                    break
                    
            if name and year and country and a_awards:
                years.append(year)
                countries.append(country)
                names.append(name)
                awards.append(a_awards)
           
    except: 
      pass

def extract_data_from_ul(ul):
    actor_urls=[]
    for li in ul.findAll('li'):
        actor_url = li.find(href=True)
        if actor_url is not None :
            actor_urls.append(base_url + actor_url.get('href'))
    for actor_url in actor_urls:
        setArraysWithActorsData(actor_url)
        
        
def extract_data_from_table(table):
    actor_urls=[]
    for row in table.findAll("tr"):
        cells=row.findAll('td')
        if(len(cells)==3 or len(cells)==2):
            actor_url = cells[0].find(href=True)
            if actor_url is not None:
                actor_urls.append(base_url + actor_url.get('href'))
        
    if len(actor_urls)==0:
       print('cells lenght problem')

    for actor_url in actor_urls:
        setArraysWithActorsData(actor_url)

    
for url in movies_url:
    print('===== ' + url)
    actor_urls=[]
    try:
        soup = getSoupFromUrl(url)
        cast_span_tag = soup.find('span', class_='mw-headline', text=re.compile('(cast)|(Cast)'))
        if cast_span_tag is None:
            continue
        header_tag= cast_span_tag.parent
        actors = header_tag.findNext(['ul','table'])
        if(actors.name=='ul'):
            extract_data_from_ul(actors)
        else:
            extract_data_from_table(actors)
    except:
      print("Problem: " + url)

    
    

    

    
    
    
    
    
df=pd.DataFrame(years,columns=['Year'])
df['Name']=names
df['Country']=countries
df['Awards']=awards

df
   

   





C:\Users\Shira\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Shira\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


===== https://en.wikipedia.org//wiki/Firehouse_(1987_film)
===== https://en.wikipedia.org//wiki/Satisfaction_(film)
===== https://en.wikipedia.org//wiki/Mystic_Pizza
===== https://en.wikipedia.org//wiki/Blood_Red_(film)
===== https://en.wikipedia.org//wiki/Steel_Magnolias
===== https://en.wikipedia.org//wiki/Pretty_Woman
===== https://en.wikipedia.org//wiki/Flatliners
===== https://en.wikipedia.org//wiki/Sleeping_with_the_Enemy
===== https://en.wikipedia.org//wiki/Dying_Young
===== https://en.wikipedia.org//wiki/Hook_(film)
===== https://en.wikipedia.org//wiki/The_Player_(1992_film)
===== https://en.wikipedia.org//wiki/The_Pelican_Brief_(film)
===== https://en.wikipedia.org//wiki/I_Love_Trouble_(1994_film)
===== https://en.wikipedia.org//wiki/Pr%C3%AAt-%C3%A0-Porter_(film)
===== https://en.wikipedia.org//wiki/Something_to_Talk_About_(film)
===== https://en.wikipedia.org//wiki/Mary_Reilly_(film)
===== https://en.wikipedia.org//wiki/Michael_Collins_(film)
===== https://en.wikipedia.org//

Year                   Name                          Country  \
0              1953          Peter Onorati              Boonton, New Jersey   
1              1961        Peter Mackenzie                           Boston   
2              1964         Craig Mitchell              Hempstead, New York   
3              1967          Julia Roberts                  Smyrna, Georgia   
4              1966        Justine Bateman                              Rye   
5              1952            Liam Neeson                        Ballymena   
6              1967         Trini Alvarado                    New York City   
7              1964           Scott Coffey                 Honolulu, Hawaii   
8              1963        Britta Phillips             Boyne City, Michigan   
9              1967          Julia Roberts                  Smyrna, Georgia   
10             1945           Debbie Harry                            Miami   
11   Year not found      Michael DeLorenzo                Country Not Found   
12             1965            Tom O'Brien              Burbank, California   
13             1941          Steve Cropper                   Dora, Missouri   
14             1971          Annabeth Gish          Albuquerque, New Mexico   
15             1967          Julia Roberts                  Smyrna, Georgia   
16             1967            Lili Taylor                          Glencoe   
17             1959      Vincent D'Onofrio            Bensonhurst, Brooklyn   
18             1959       William R. Moses          Los Angeles, California   
19             1962            Adam Storke               New York, New York   
20             1943       Conchata Ferrell                       Loudendale   
21             1931          Joanna Merlin                          Chicago   
22             1933          Louis Turenne                         Montreal   
23   Year not found           Janet Zarish                          Chicago   
24             1934              Ann Flood                Country Not Found   
25             1970             Matt Damon         Cambridge, Massachusetts   
26             1956           Eric Roberts              Biloxi, Mississippi   
27             1942     Giancarlo Giannini                        La Spezia   
28             1936          Dennis Hopper               Dodge City, Kansas   
29             1940             Burt Young                           Queens   
..              ...                    ...                              ...   
689            1976       Gabriel Iglesias                        San Diego   
690            1962            Jeff Dunham                    Dallas, Texas   
691            1960           Kelly Asbury                  Beaumont, Texas   
692            1946           Frank Welker                 Denver, Colorado   
693            1962      Dee Bradley Baker             Bloomington, Indiana   
694            1967          Julia Roberts                  Smyrna, Georgia   
695            1968            Owen Wilson                           Dallas   
696            2006         Jacob Tremblay                        Vancouver   
697            2001        Izabela Vidovic                          Chicago   
698            1950            Sônia Braga                          Maringá   
699            1952         Mandy Patinkin                          Chicago   
700            1982           Daveed Diggs              Oakland, California   
701            1999  Danielle Rose Russell  Pequannock Township, New Jersey   
702            1996            Nadji Jeter                 Atlanta, Georgia   
703            2005              Noah Jupe                        Islington   
704            2006           Millie Davis                          Toronto   
705            1970          Nicole Oliver                           Ottawa   
706            1981           Crystal Lowe                        Vancouver   
707            1965            Steve Bacic                       